In [39]:
# %load init.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('../data/complaints.csv')

In [40]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
4,2019-02-08,Vehicle loan or lease,Lease,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,NaN,NaN,HYUNDAI CAPITAL AMERICA,FL,33161,NaN,Consent not provided,Web,2019-02-08,Closed with non-monetary relief,Yes,NaN,3146310


In [41]:
df['Date'] = pd.to_datetime(df['Date received'],infer_datetime_format=True)
df = df.rename(columns={'Complaint ID': 'All_Issues'})
df = df.rename(columns={'Timely response?': 'Responsive'})
dfg = df.groupby(['Responsive','Company']).count()[['All_Issues']].sort_values('All_Issues', ascending=False)
dfg = dfg.reset_index()

In [42]:
nBanks = 100
dfgTopN = dfg.head(nBanks) 
dfgTopN['Company']
dfgAppend = dfg[nBanks:len(dfg.index)] # the other banks that didn't make the topN
dfgAppend = pd.DataFrame({"Company":['Other'],
                          "All_Issues": dfgOther['All_Issues'].sum()})
dfgTopN.append(dfgAppend, sort=True, ignore_index=True)

,All_Issues,Company,Responsive
0,151153,"EQUIFAX, INC.",Yes
1,135717,Experian Information Solutions Inc.,Yes
2,132192,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Yes
3,87090,"BANK OF AMERICA, NATIONAL ASSOCIATION",Yes
4,72928,WELLS FARGO & COMPANY,Yes
...,...,...,...
96,1446,CORELOGIC INC,Yes
97,1417,AMERICAN HONDA FINANCE CORP,Yes
98,1415,HYUNDAI CAPITAL AMERICA,Yes
99,1407,RoundPoint Mortgage Servicing Corporation,Yes


In [43]:
dfpivot = dfgTopN.pivot(index='Responsive', columns='Company', values='All_Issues')
dfpivot = dfpivot.reset_index()
dfpivot = dfpivot.fillna(0)
dfpivot

Company,Responsive,AES/PHEAA,AFNI INC.,ALLY FINANCIAL INC.,AMERICAN EXPRESS COMPANY,AMERICAN HONDA FINANCE CORP,Ad Astra Recovery Services Inc,Alliance Data Card Services,Allied Interstate LLC,"BANK OF AMERICA, NATIONAL ASSOCIATION",...,Specialized Loan Servicing Holdings LLC,TCF NATIONAL BANK,TD BANK US HOLDING COMPANY,TOYOTA MOTOR CREDIT CORPORATION,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",TRANSWORLD SYSTEMS INC,U.S. BANCORP,UNITED SERVICES AUTOMOBILE ASSOCIATION,WELLS FARGO & COMPANY,"WESTERN UNION COMPANY, THE"
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,No,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1590.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3697.0,0.0
2,Yes,10312.0,2863.0,6474.0,15962.0,1417.0,1763.0,7808.0,1608.0,87090.0,...,5098.0,1947.0,11266.0,2106.0,132192.0,5127.0,19134.0,6660.0,72928.0,2679.0


In [44]:
#dfpivot.loc[:,dfg['Company']].plot.bar(stacked=True)